# Computer Vision

## Overview

* Lochkamera
* Farbraeume
* CCD/CMOS
* Weltkoordinaten
* Bayersfilter
* RGB / HSV
* Camshift
* Image moments
    * Ableitung
* Background Subtraction
    * Exponentieller Tiefpass
* Morphologische Operatoren
    * Strukturelemente
    * Erosion
    * Dilatation
    * Opening & Closing
* Sobel
* Canny Edge
    * Non-maximum suppression
* HOG
    * Invariant gegen scale und translation
* Hough Transformation
* Interest Points
* Taylor approximation
* Moravecs Corners
* Harris Corner Detection
    * Structure Tensor
* LoG und DoG
* Optical Flow
* Lucas-Kanade
* Shi-Tomasi
* SURF
* SIFT



In [1]:
# do all the import stuff
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image
from scipy import signal, ndimage
import numpy as np
import math

# import my modules
import sys
sys.path.append('/home/julian/dev/computer_vision_ws2015/')

import juvis as jv